In [1]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, GRPO_CONNECTIONS_CONFIG

cfg = GRPO_CONNECTIONS_CONFIG
cfg.train_batch_size = 2
cfg.num_generations = 2
# cfg.model_id_or_path = SMOL_LM_135M
cfg.notebook_mode = True
cfg.max_samples = 1000

/home/brianf/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-28 21:11:19,734	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
wrapper = TrainerWrapper(cfg)
wrapper.init_model()

2025-03-28 21:11:26.637 | INFO     | trl_wrapper.trainer_wrapper:__init__:264 - Setting padding side to: left
2025-03-28 21:11:26.722 | INFO     | trl_wrapper.trainer_wrapper:init_model:275 - Using device: cuda:0
2025-03-28 21:11:26.724 | INFO     | trl_wrapper.trainer_wrapper:init_model:289 - Loading model Qwen/Qwen2.5-1.5B-Instruct with attn_impl: flash_attention_2


In [3]:
wrapper.init_data_module()

2025-03-28 21:13:28.051 | INFO     | trl_wrapper.wrapper_config:__init__:133 - Cache dir: ../dataset_caches/connections_data_module
Map: 100%|██████████| 353/353 [00:00<00:00, 8064.39 examples/s]


In [ ]:
wrapper.data_module.train_dataset[0]

In [4]:
wrapper.init_trainer()

2025-03-28 21:13:30.151 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:352 - Saving output to: ./runs/03-28-21-13-71503-qwen2.5-1.5b-instruct-
2025-03-28 21:13:30.159 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:375 - Initializing trainer, run_name: 03-28-21-13-71503-qwen2.5-1.5b-instruct-, wandb project: qwen-connections-grpo
2025-03-28 21:13:30.160 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:460 - Using vllm: False


In [5]:
from model.reasoning import GSM8K_SYSTEM_PROMPT, CONNECTIONS_PROMPT
from model.utils import get_available_device

messages = [
    {"role": "system", "content": CONNECTIONS_PROMPT},
    {
        "role": "user",
        "content": "train, panda, dove, series, wind, bear, orca, bass, string, skunk, speed, sand, zebra, tourist, desert, chain",
    },
]
tokenized_chat = wrapper.tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
    return_attention_mask=True,
)

device = get_available_device()
tokenized_chat = tokenized_chat.to(device)
out = wrapper.model.generate(tokenized_chat, max_length=1024)
print(wrapper.tokenizer.decode(out[0], skip_special_tokens=True))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


system

You are an expert puzzle solving model.
Find groups of words that are related to each other. Each group is four words long.
You may only use each word in one group.
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
<group>
...
</group>
<group>
...
</group>
</answer>

# Example

User: apple, orange, banana, pear, corolla, charger,
Assistant: <reasoning>
The first group are all fruits.
The second group are all cars.
</reasoning>
<answer>
<group>apple, orange, banana, pear</group>
<group>corolla, charger</group>
</answer>

# Example

User: dog, cat, red, white,
Assistant: <reasoning>
The first group are all animals.
The second group are all colors.
</reasoning>
<answer>
<group>dog, cat</group>
<group>red, white</group>
</answer>

user
train, panda, dove, series, wind, bear, orca, bass, string, skunk, speed, sand, zebra, tourist, desert, chain
assistant
<reasoning>
The first group can be classified as vehicles and animals: train, panda, dove, bear, orca, and sk

In [8]:
wrapper.train()

2025-03-28 21:14:57.826 | INFO     | trl_wrapper.trainer_wrapper:train:624 - Starting training.
2025-03-28 21:15:02.163 | INFO     | model.reasoning:xmlcount_reward_func:61 - XML count rewards: [0.192, 0.241]
2025-03-28 21:15:02.165 | INFO     | model.reasoning:strict_format_reward_func:107 - Strict format rewards: [0.0, 0.0]
2025-03-28 21:15:02.166 | INFO     | model.reasoning:connections_soft_group_reward_func:180 - Connections func rewards: [[['construction', 'whooping', 'big', 'euphoria'], ['giant', 'humor', 'glee', 'felicity', 'huge', 'travel', 'cheers', 'paper', 'poetry', 'fiction', 'great', 'frasier']], [['construction', 'travel'], ['whooping', 'glee', 'felicity', 'cheers', 'paper', 'poetry', 'fiction', 'great', 'frasier']]]
2025-03-28 21:15:02.168 | INFO     | model.reasoning:connections_soft_group_reward_func:182 - Connections scores: [0.0, 0.0]
2025-03-28 21:15:02.169 | INFO     | model.reasoning:group_size_reward_func:191 - Group size rewards: [0.0, 0.0]
2025-03-28 21:15:02.

Generations:
--------------------------------------------------------------------------------
<reasoning>
The first group are all words related to construction.
The second group are all words related to emotions.
</reasoning>
<answer>
<group>construction, whooping, big, euphoria</group>
<group>giant, humor, glee, felicity, huge, travel, cheers, paper, poetry, fiction, great, frasier</group>
</answer>
--------------------------------------------------------------------------------
<reasoning>
The first group are all related to construction.
The second group are all related to travel.
</reasoning>
<answer>
<group>construction, travel</group>
<group>whooping, glee, felicity, cheers, paper, poetry, fiction, great, frasier</group>
</answer>


2025-03-28 21:15:05.834 | INFO     | model.reasoning:xmlcount_reward_func:61 - XML count rewards: [0.264, 0.22999999999999998]
2025-03-28 21:15:05.835 | INFO     | model.reasoning:strict_format_reward_func:107 - Strict format rewards: [0.0, 0.0]
2025-03-28 21:15:05.836 | INFO     | model.reasoning:connections_soft_group_reward_func:180 - Connections func rewards: [[['ring', 'lion', 'joker', 'wardrobe'], ['clown', 'smoker', 'scarecrow', 'sinner']], [['ring', 'lion', 'joker', 'wardrobe'], ['props', 'hair', 'lover', 'wizard', 'trapeze', 'clown', 'smoker', 'scarecrow', 'sinner']]]
2025-03-28 21:15:05.836 | INFO     | model.reasoning:connections_soft_group_reward_func:182 - Connections scores: [0.0, 0.0]
2025-03-28 21:15:05.837 | INFO     | model.reasoning:group_size_reward_func:191 - Group size rewards: [0.0, 0.0]
2025-03-28 21:15:05.838 | INFO     | model.reasoning:n_groups_reward_func:199 - Number of groups rewards: [0.0, 0.0]


Generations:
--------------------------------------------------------------------------------
<reasoning>
The first group are all items used in a circus.
The second group are all characters from a fairy tale.
</reasoning>
<answer>
<group>ring, lion, joker, wardrobe</group>
<group>clown, smoker, scarecrow, sinner</group>
</answer>
--------------------------------------------------------------------------------
<reasoning>
The first group are all characters from a circus.
The second group are all items related to makeup.
</reasoning>
<answer>
<group>ring, lion, joker, wardrobe</group>
<group>props, hair, lover, wizard, trapeze, clown, smoker, scarecrow, sinner</group>
</answer>


2025-03-28 21:15:08.567 | INFO     | model.reasoning:xmlcount_reward_func:61 - XML count rewards: [0.299, 0.29100000000000004]
2025-03-28 21:15:08.568 | INFO     | model.reasoning:strict_format_reward_func:107 - Strict format rewards: [0.0, 0.0]
2025-03-28 21:15:08.569 | INFO     | model.reasoning:connections_soft_group_reward_func:180 - Connections func rewards: [[['hold', 'maintain', 'chair', 'couch'], ['goldilocks', 'balanced', 'bed', 'table']], [['hold', 'maintain', 'chair', 'bear'], ['sweet', 'dry', 'balanced', 'full', 'goldilocks', 'table']]]
2025-03-28 21:15:08.570 | INFO     | model.reasoning:connections_soft_group_reward_func:182 - Connections scores: [0.0, 0.0]
2025-03-28 21:15:08.571 | INFO     | model.reasoning:group_size_reward_func:191 - Group size rewards: [0.0, 0.0]
2025-03-28 21:15:08.571 | INFO     | model.reasoning:n_groups_reward_func:199 - Number of groups rewards: [0.0, 0.0]


Generations:
--------------------------------------------------------------------------------
<reasoning>
The first group are all furniture.
The second group are all words.
</reasoning>
<answer>
<group>hold, maintain, chair, couch</group>
<group>goldilocks, balanced, bed, table</group>
</answer>
--------------------------------------------------------------------------------
<reasoning>
The first group are all nouns.
The second group are all verbs.
</reasoning>
<answer>
<group>hold, maintain, chair, bear</group>
<group>sweet, dry, balanced, full, goldilocks, table</group>
</answer>


KeyboardInterrupt: 